In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_fold_1_train = pd.read_csv('../results/fold_1/train_preds.csv')
df_fold_2_train = pd.read_csv('../results/fold_2/train_preds.csv')
df_fold_3_train = pd.read_csv('../results/fold_3/train_preds.csv')
df_fold_4_train = pd.read_csv('../results/fold_4/train_preds.csv')
df_fold_0_train = pd.read_csv('../results/fold_0/train_preds.csv')


df_fold_1_valid = pd.read_csv('../results/fold_1/valid_preds.csv')
df_fold_2_valid = pd.read_csv('../results/fold_2/valid_preds.csv')
df_fold_3_valid = pd.read_csv('../results/fold_3/valid_preds.csv')
df_fold_4_valid = pd.read_csv('../results/fold_4/valid_preds.csv')
df_fold_0_valid = pd.read_csv('../results/fold_0/valid_preds.csv')

In [3]:
df_fold = pd.concat([df_fold_1_train, df_fold_2_train])
df_fold = pd.concat([df_fold, df_fold_3_train])
df_fold = pd.concat([df_fold, df_fold_4_train])
df_fold = pd.concat([df_fold, df_fold_0_train])


In [4]:
df_train = df_fold.groupby('index').agg('mean').reset_index()

In [5]:
p = re.compile('label_*')
label_cols = [ s for s in df_train.columns if p.match(s) ]

def get_original_label(row):
    
    for label, item in zip(label_cols, row[label_cols]):
        if item == 1:
            return label

In [6]:
df_train['original_label']  = df_train.apply(get_original_label, axis=1)

In [7]:
pred_cols = [label.split('_')[-1] for label in label_cols]

In [8]:
def get_predicted_label(row):
    local_row = row[pred_cols]
    indx = np.argmax(local_row)
    conf = local_row[indx]
    row['confidence'] = conf
    row['pred_label'] = indx
    return row
    
#     for label, item in zip(label_cols, row[label_cols]):
#         if item == 1:
#             return label

In [9]:
df_train['pred_label'] = -1
df_train['confidence'] = -1

In [10]:
df_train = df_train.apply(get_predicted_label, axis=1)

In [11]:
def get_training_accuracy(row):
    if row['original_label'] == 'label_'+str(row['pred_label']):
        return 'yes'
    else:
        return 'no'

In [12]:
df_train['correct'] = df_train.apply(get_training_accuracy, axis=1)

In [13]:
df_train['correct'].value_counts()

yes    328
Name: correct, dtype: int64

## Validation Accuracy

In [14]:
df_fold = pd.concat([df_fold_1_valid, df_fold_2_valid])
df_fold = pd.concat([df_fold, df_fold_3_valid])
df_fold = pd.concat([df_fold, df_fold_4_valid])
df_fold = pd.concat([df_fold, df_fold_0_valid])


In [15]:
df_valid = df_fold
df_valid

index                            sentence  fold  label_0  label_1  \
0       3          can pay later on delivery      1        0        1   
1      27                             order #     1        0        0   
2      28            what is my order status?     1        0        0   
3      35  what are the sof mattress products     1        0        0   
4      48         want to know about warranty     1        0        0   
..    ...                                 ...   ...      ...      ...   
61    304                        order status     0        0        0   
62    306                              outlet     0        0        0   
63    315                              inches     0        0        0   
64    321        tell me about sof mattresses     0        0        0   
65    327    will you be able to deliver here     0        0        0   

    label_2  label_3  label_4  label_5  label_6  ...        11        12  \
0         0        0        0        0        0  ...  0.002779  0.000950   
1         0        0        0        0        0  ...  0.000696  0.000555   
2         0        0        0        0        0  ...  0.000759  0.000418   
3         0        0        0        0        0  ...  0.000390  0.099681   
4         0        0        0        1        0  ...  0.000104  0.001087   
..      ...      ...      ...      ...      ...  ...       ...       ...   
61        0        0        0        0        0  ...  0.000320  0.000053   
62        0        0        0        0        0  ...  0.993782  0.001248   
63        0        0        0        0        0  ...  0.000100  0.000043   
64        0        0        0        0        0  ...  0.000058  0.000521   
65        0        0        0        0        0  ...  0.000009  0.000005   

          13        14        15        16        17        18        19  \
0   0.002196  0.002424  0.007956  0.003878  0.576895  0.074877  0.000757   
1   0.000169  0.000108  0.000751  0.989363  0.000236  0.002638  0.000135   
2   0.000155  0.000091  0.000710  0.989602  0.000238  0.002747  0.000120   
3   0.717535  0.132713  0.002841  0.001044  0.000374  0.000398  0.002304   
4   0.000837  0.000331  0.000222  0.001293  0.000338  0.001010  0.000409   
..       ...       ...       ...       ...       ...       ...       ...   
61  0.000059  0.000011  0.000199  0.997881  0.000051  0.000212  0.000103   
62  0.001045  0.000115  0.000014  0.000917  0.000026  0.000025  0.000397   
63  0.000255  0.000011  0.000015  0.000677  0.000012  0.000099  0.000055   
64  0.005032  0.991593  0.000039  0.000037  0.000054  0.000026  0.000231   
65  0.000084  0.000009  0.000117  0.000110  0.000478  0.000026  0.000455   

          20  
0   0.146063  
1   0.000154  
2   0.000178  
3   0.000173  
4   0.000044  
..       ...  
61  0.000052  
62  0.000224  
63  0.000009  
64  0.000511  
65  0.000002  

[328 rows x 45 columns]

In [16]:
df_valid['original_label']  = df_valid.apply(get_original_label, axis=1)

In [17]:
df_valid['pred_label'] = -1
df_valid['confidence'] = -1

In [18]:
df_valid = df_valid.apply(get_predicted_label, axis=1)

In [19]:
df_valid['correct'] = df_valid.apply(get_training_accuracy, axis=1)

In [20]:
df_valid.correct.value_counts(normalize=True) * 100

yes    90.54878
no      9.45122
Name: correct, dtype: float64

## Out of fold validation accuracy is 90%

### Fold wise validation accuracy

In [23]:
for i in range(0,5):
    print("Fold ", i)
    print(df_valid[df_valid.fold == i].correct.value_counts(normalize =True)*100)

Fold  0
yes    87.878788
no     12.121212
Name: correct, dtype: float64
Fold  1
yes    89.393939
no     10.606061
Name: correct, dtype: float64
Fold  2
yes    90.909091
no      9.090909
Name: correct, dtype: float64
Fold  3
yes    92.307692
no      7.692308
Name: correct, dtype: float64
Fold  4
yes    92.307692
no      7.692308
Name: correct, dtype: float64


In [24]:
for i in range(0,5):
    print("Fold ", i)
    print(df_valid[df_valid.fold == i].correct.value_counts())

Fold  0
yes    58
no      8
Name: correct, dtype: int64
Fold  1
yes    59
no      7
Name: correct, dtype: int64
Fold  2
yes    60
no      6
Name: correct, dtype: int64
Fold  3
yes    60
no      5
Name: correct, dtype: int64
Fold  4
yes    60
no      5
Name: correct, dtype: int64


In [25]:
df_valid.fold.value_counts()

2    66
1    66
0    66
4    65
3    65
Name: fold, dtype: int64